In [ ]:
!pip install llama-index-core
!pip install llama-index-llms-ollama
!pip install llama-index-embeddings-ollama
!pip install llama-index-utils-workflow
!pip install llama-parse

In [1]:
# llama-parse is async-first, running the async code in a notebook requires the use of nest_asyncio
import os
import nest_asyncio

nest_asyncio.apply()

In [1]:
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import Settings

# Connect to Ollama running in Docker (exposed on port 11434)
llm = Ollama(model="qwen2.5:7b", base_url="http://10.200.2.34:32411")

messages = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content="You are a helpful assistant that answers in one sentence.",
    ),
    ChatMessage(
        role=MessageRole.USER,
        content="Xin chào, hãy giới thiệu về bạn",
    ),
]

response = llm.chat(messages)
print(response)

# Configure settings to use Ollama
Settings.llm = Ollama(model="qwen2.5:7b", base_url="http://10.200.2.34:32411", request_timeout=3000)
Settings.embed_model = OllamaEmbedding(model_name='mxbai-embed-large:335m', base_url="http://10.200.2.34:32411", request_timeout=3000)


assistant: Xin chào, tôi là một trợ lý AI có thể giúp bạn với nhiều loại câu hỏi và tác vụ khác nhau.


In [3]:
from qdrant_client import AsyncQdrantClient

# Kết nối với Qdrant
async_client = AsyncQdrantClient("localhost", port=6333)

async def get_collection_info():
    collection_info = await async_client.get_collection("llamaindex_collection")
    print(collection_info)

# Run the async function
import asyncio
asyncio.run(get_collection_info())

status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1566 segments_count=8 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=No

In [4]:
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex

collection_name = "labour_collection"

# 2. Tạo Qdrant Vector Store từ collection đã tồn tại
vector_store = QdrantVectorStore(
    aclient=async_client,
    collection_name=collection_name
)

# 3. Tạo VectorStoreIndex từ Vector Store
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

# (Tùy chọn) Kiểm tra hoặc sử dụng index
query_engine = index.as_query_engine(similarity_top_k=1)
# response = query_engine.query("What was San Francisco's budget for Police Department in 2023?")
# print(response)

In [5]:
from llama_index.core.tools import QueryEngineTool

law_tool = QueryEngineTool.from_defaults(
    query_engine,
    name="labor_law_tool", # <--- Changed name
    description="Công cụ truy vấn thông tin chi tiết về các quy định pháp luật lao động tại Việt Nam, bao gồm Bộ luật Lao động, Luật Việc làm, Luật An toàn vệ sinh lao động, và các văn bản liên quan.", # <--- Updated description
)


# budget_tool = QueryEngineTool.from_defaults(
#     query_engine,
#     name="san_francisco_budget_2023",
#     description="A RAG engine with extremely detailed information about the 2023 San Francisco budget.",
# )

In [ ]:
from typing import Union
from llama_index.core.workflow import step, Workflow, Event, Context, StartEvent, StopEvent
from collections import defaultdict
from llama_index.core.agent import FunctionCallingAgent
import json

# === Define Events ===

class ContractFileEvent(StartEvent):
    file_path: str
    query: str
    tools: list

class ContractTextEvent(Event):
    contract_text: str

class ContextAnalyzedEvent(Event):
    contract_type: str
    key_elements: list
    # legal_domain: list
    cited_legal_documents: list
    contract_text: str

class AnalysisResultsEvent(Event):
    legal_issues: str
    logic_issues: str
    risk_issues: str

class FinalOutputEvent(Event):
    report: str
    # annotations: str

class ProgressEvent(Event):
    progress: str

# === Define Workflow ===
class MultiAgentContractReviewWorkflow(Workflow):

    @step()
    async def preprocess_input(self, ctx: Context, ev: StartEvent) -> ContractTextEvent:
        
        await ctx.set("query", ev.query)
        await ctx.set("tools", ev.tools)
        await ctx.set("contract_text", ev.file_path)  # file_path now contains plain contract text

        ctx.write_event_to_stream(ProgressEvent(progress="Preprocessing completed."))
        return ContractTextEvent(contract_text=ev.file_path)
      
    @step()
    async def analyze_context(self, ctx: Context, ev: ContractTextEvent) -> ContextAnalyzedEvent:
        query = await ctx.get("query")
        contract_text = ev.contract_text

        prompt = f"""
        **Yêu cầu:** Dựa **chặt chẽ và duy nhất** vào nội dung hợp đồng dưới đây và yêu cầu kiểm tra '{query}', hãy thực hiện các việc sau:
        1.  **Xác định loại hợp đồng:** Trích xuất **chính xác** tên loại hợp đồng **được nêu rõ** trong văn bản. Nếu không nêu rõ, hãy ghi "Không xác định rõ trong văn bản".
        2.  **Trích xuất yếu tố/điều khoản chính:** Liệt kê các yếu tố hoặc điều khoản **được đề cập trực tiếp** trong văn bản hợp đồng (ví dụ: các bên tham gia, đối tượng hợp đồng, giá trị/tiền lương, thời hạn, quyền và nghĩa vụ cơ bản, điều khoản thanh toán, điều khoản chấm dứt, các điều luật được trích dẫn). Chỉ liệt kê những gì có trong văn bản.
        3.  **Xác định văn bản luật được trích dẫn:** Liệt kê **tên các văn bản luật hoặc nghị định** (ví dụ: Bộ luật Lao động 2019, Luật BHXH 2014, Nghị định 38/2022/NĐ-CP) được **trích dẫn trực tiếp** trong nội dung hợp đồng.

        **Nghiêm cấm:** Không được đưa ra bất kỳ giả định, suy diễn hoặc thông tin nào không được nêu rõ trong văn bản hợp đồng dưới đây.

        **Nội dung hợp đồng:**
        ---
        {contract_text}
        ---

        **Định dạng trả lời:** Chỉ trả lời bằng định dạng JSON sau, không thêm bất kỳ giải thích nào khác. Nếu một trường thông tin không thể xác định từ văn bản, hãy để giá trị là `null` hoặc danh sách rỗng `[]`.
        {{
          "contract_type": "...",
          "key_elements": ["...", "..."],
          "cited_legal_documents": ["...", "..."]
        }}
        """

        response = await Settings.llm.acomplete(prompt)
        
        print(response)
        
        parsed = json.loads(str(response))
        
        # print(parsed)
        # contract_type=parsed["contract_type"]
        # key_elements=parsed["key_elements"]
        # legal_domain=parsed["potential_legal_areas"]
        # print(f"Contract Type: {contract_type}")
        # print(f"Key Elements: {key_elements}")
        # print(f"Legal Domain: {legal_domain}")
        
        contract_type = parsed.get("contract_type")
        key_elements = parsed.get("key_elements", [])
        cited_laws = parsed.get("cited_legal_documents", [])
        
        await ctx.set("contract_type", contract_type)
        await ctx.set("key_elements", key_elements)
        await ctx.set("cited_laws", cited_laws)

        ctx.write_event_to_stream(ProgressEvent(progress="Context analysis completed."))
        return ContextAnalyzedEvent(
            contract_type=contract_type,
            key_elements=key_elements,
            cited_legal_documents=cited_laws,
            contract_text=contract_text,
        )

    @step()
    async def detailed_analysis(self, ctx: Context, ev: ContextAnalyzedEvent) -> AnalysisResultsEvent:
        contract_text = ev.contract_text
        query = await ctx.get("query")

        contract_type = ev.contract_type
        cited_laws_list = ev.cited_legal_documents
        
        logic_prompt = f"""
        **Yêu cầu:** Phân tích **chỉ dựa trên văn bản hợp đồng** sau đây để xác định các vấn đề về logic và ngữ nghĩa **có bằng chứng trực tiếp** trong văn bản:
        1.  **Mâu thuẫn nội dung:** Tìm các điều khoản **có nội dung trái ngược trực tiếp** với nhau trong văn bản.
        2.  **Ngôn ngữ mơ hồ:** Xác định các từ ngữ, cụm từ hoặc điều khoản **không rõ ràng về mặt ngữ nghĩa**, có thể dẫn đến nhiều cách hiểu khác nhau **ngay trong ngữ cảnh của hợp đồng**.
        3.  **Thiếu nhất quán:** Tìm sự thiếu nhất quán trong việc sử dụng thuật ngữ, định nghĩa hoặc cấu trúc câu **trong chính văn bản hợp đồng**.

        **Nghiêm cấm:** Không đưa ra bất kỳ giả định, suy diễn hoặc đánh giá nào không dựa trên bằng chứng trực tiếp từ văn bản hợp đồng. Chỉ tập trung vào các vấn đề logic và ngữ nghĩa nội tại của văn bản.

        **Thông tin tham khảo (chỉ để hiểu ngữ cảnh, không dùng để suy diễn):**
        - Loại hợp đồng đã xác định: {contract_type}
        - Bối cảnh/Yêu cầu ban đầu: {query}

        **Nội dung hợp đồng cần phân tích:**
        ---
        {contract_text}
        ---

        **Định dạng trả lời:** Chỉ trả lời bằng định dạng JSON sau. Nếu không tìm thấy vấn đề nào thuộc các loại trên, trả về danh sách `logic_findings` rỗng `[]`.
        {{
          "logic_findings": [
            {{"type": "contradiction/ambiguity/inconsistency", "description": "Mô tả vấn đề dựa trên văn bản", "clauses_involved": ["Trích dẫn chính xác điều khoản/phần văn bản liên quan 1", "Trích dẫn chính xác điều khoản/phần văn bản liên quan 2 (nếu có)"]}},
            ...
          ]
        }}
        """

        risk_prompt = f"""
        **Yêu cầu:** Dựa **chỉ vào nội dung văn bản hợp đồng** dưới đây và loại hợp đồng '{contract_type}', hãy thực hiện các việc sau:
        1.  **Kiểm tra sự hiện diện của điều khoản:** Xác định xem các điều khoản thường gặp sau đây có **xuất hiện trong văn bản hợp đồng hay không**: phạt vi phạm, bảo mật, bất khả kháng, luật áp dụng, giải quyết tranh chấp. Chỉ liệt kê những điều khoản **không tìm thấy** trong văn bản.
        2.  **Xác định điều khoản có thể bất lợi (dựa trên từ ngữ):** Tìm các điều khoản mà **cách diễn đạt trong văn bản** có vẻ tạo ra sự mất cân bằng rõ rệt về quyền lợi/nghĩa vụ giữa các bên. Trích dẫn cụ thể điều khoản đó.
        3.  **Xác định rủi ro từ nội dung hiện có:** Tìm các rủi ro **phát sinh trực tiếp từ cách các điều khoản hiện có được viết** (ví dụ: nghĩa vụ không rõ ràng, thời hạn không cụ thể).

        **Nghiêm cấm:** Không được đưa ra bất kỳ giả định, suy diễn về rủi ro không dựa trên bằng chứng trực tiếp từ văn bản. Không đánh giá điều khoản là "quan trọng" hay "không quan trọng" trừ khi văn bản tự nêu vậy. Không đề xuất khắc phục dựa trên kiến thức bên ngoài, chỉ đề xuất dựa trên việc làm rõ hoặc sửa đổi các điểm đã xác định **ngay trong văn bản**.

        **Thông tin tham khảo (chỉ để hiểu ngữ cảnh, không dùng để suy diễn):**
        - Yêu cầu ban đầu: {query}

        **Nội dung hợp đồng cần phân tích:**
        ---
        {contract_text}
        ---

        **Định dạng trả lời:** Chỉ trả lời bằng định dạng JSON sau. Nếu không tìm thấy rủi ro nào thuộc các loại trên, trả về danh sách `risk_findings` rỗng `[]`. Đối với đề xuất, chỉ tập trung vào việc làm rõ/sửa đổi dựa trên vấn đề đã xác định từ văn bản.
        {{
          "risk_findings": [
            {{"risk_type": "missing_clause/potentially_unfavorable_clause/risk_from_wording", "description": "Mô tả rủi ro dựa trên văn bản...", "suggestion": "Đề xuất làm rõ/sửa đổi dựa trên vấn đề..."}},
            ...
          ]
        }}
        """

        legal_prompt = f"""
        **Yêu cầu:** Xem xét **kỹ lưỡng từng điều khoản** của văn bản hợp đồng dưới đây, đặc biệt chú ý đến các **trích dẫn pháp luật** (ví dụ: Điều X, Luật Y). Dựa **chỉ vào nội dung văn bản hợp đồng và danh sách các luật được trích dẫn ({cited_laws_list})**, hãy xác định các điểm cụ thể mà:

        a)  Nội dung điều khoản hợp đồng **có vẻ không nhất quán, xung đột, hoặc diễn giải sai** ý nghĩa của chính điều luật/nghị định mà nó trích dẫn.
        b)  Việc **trích dẫn pháp luật có vẻ không chính xác hoặc không tồn tại** (ví dụ: số điều/khoản không hợp lý, tên luật không khớp với nội dung điều khoản).
        c)  Điều khoản **thiếu thông tin hoặc quy định chi tiết** mà luật/nghị định được trích dẫn thường yêu cầu.
        d)  Điều khoản sử dụng **từ ngữ hoặc quy định có thể gây tranh cãi pháp lý** khi đối chiếu với chuẩn mực chung, ngay cả khi không có trích dẫn cụ thể.

        **Nhiệm vụ của bạn là xác định các điểm "cờ đỏ" (red flags) liên quan đến pháp lý dựa trên văn bản, bao gồm cả tính chính xác của các trích dẫn, chứ KHÔNG phải đưa ra kết luận pháp lý cuối cùng.**

        **Nghiêm cấm:**
        *   **KHÔNG liệt kê mọi điều khoản có trích dẫn luật một cách tự động.** Chỉ nêu ra nếu có dấu hiệu (a), (b), (c), hoặc (d).
        *   **KHÔNG tự tạo ra các truy vấn về điều luật không liên quan đến nội dung HĐ.**
        *   **KHÔNG đưa ra giả định** hoặc thông tin bên ngoài văn bản.

        **Nội dung hợp đồng cần xem xét:**
        ---
        {contract_text}
        ---

        **Định dạng trả lời:** Liệt kê các điểm cần kiểm tra pháp lý dưới dạng danh sách. Mỗi điểm cần nêu rõ:
        - **Điều khoản/Nội dung HĐ:** (Trích dẫn ngắn gọn phần văn bản có vấn đề, bao gồm cả trích dẫn luật nếu có)
        - **Lý do cần kiểm tra:** (Giải thích ngắn gọn tại sao nó là 'cờ đỏ' - ví dụ: 'Nội dung có vẻ mâu thuẫn với Điều X Luật Y được trích dẫn', **'Trích dẫn Điều X Luật Y có vẻ không chính xác/không tồn tại'**, 'Thiếu chi tiết Z theo Luật A', 'Cách diễn đạt W có thể gây tranh cãi').

        Nếu **hoàn toàn không xác định được điểm nào** cần kiểm tra dựa trên các tiêu chí trên trong văn bản, trả về **chính xác** chuỗi: "Không xác định được điểm cụ thể nào trong văn bản cần đánh giá pháp lý thêm."
        """

        logic_response = await Settings.llm.acomplete(logic_prompt)
        risk_response = await Settings.llm.acomplete(risk_prompt)
        legal_review_response = await Settings.llm.acomplete(legal_prompt)
        
        print("------------------------------------------------------")
        print(legal_review_response)
        print("------------------------------------------------------")

        logic_issues_text = logic_response.text
        risk_issues_text = risk_response.text
        legal_review_text = legal_review_response.text

        # --- Logic xử lý kết quả phân tích pháp lý và RAG (Cải tiến cốt lõi) ---
        final_legal_issues_output = ""
        legal_issues_results = []
        no_issues_found_msg = "Không xác định được điểm cụ thể nào trong văn bản cần đánh giá pháp lý thêm."

        # <<< CHANGE: Sửa lỗi kiểm tra và xử lý khi không có điểm pháp lý >>>
        if legal_review_text.strip() == no_issues_found_msg:
            final_legal_issues_output = no_issues_found_msg
            print("--- LLM xác định không có điểm pháp lý cần tra cứu thêm ---")
        else:
            agent = FunctionCallingAgent.from_tools(
                await ctx.get("tools"),
                verbose=True, # Giữ verbose để debug
            )

            # <<< CHANGE: Phân tích cú pháp output của legal_prompt >>>
            potential_issues = []
            current_issue = {}
            lines = [line.strip() for line in legal_review_text.split('\n') if line.strip()]
            for line in lines:
                
                 print(line)
                 print('111111111111111111111111111111111')
                
                 # Đơn giản hóa việc trích xuất, giả định mỗi cặp dòng là một vấn đề
                 if line.startswith("- **Điều khoản/Nội dung HĐ:"):
                     if current_issue: potential_issues.append(current_issue)
                     current_issue = {"clause_content": line.replace("- **Điều khoản/Nội dung HĐ:**", "").strip()}
                 elif line.startswith("- **Lý do cần kiểm tra:") and current_issue:
                     current_issue["reason_for_check"] = line.replace("- **Lý do cần kiểm tra:**", "").strip()
                     # Giả định lý do kết thúc một vấn đề, thêm vào danh sách
                     potential_issues.append(current_issue)
                     current_issue = {} # Reset cho vấn đề tiếp theo
            if current_issue and "clause_content" in current_issue and "reason_for_check" not in current_issue:
                 # Xử lý trường hợp dòng cuối là clause_content mà không có reason theo sau
                 current_issue["reason_for_check"] = "Lý do không được nêu rõ"
                 potential_issues.append(current_issue)
            elif current_issue and "clause_content" in current_issue : # Handle last issue if reason was present
                 potential_issues.append(current_issue)


            print(f"--- Các điểm pháp lý LLM xác định cần kiểm tra: {len(potential_issues)} ---")

            for issue in potential_issues:
                
                print(issue)
                
                clause_content = issue.get("clause_content", "N/A")
                reason = issue.get("reason_for_check", "N/A")

                # <<< CHANGE: Tạo query RAG thông minh hơn >>>
                query_for_rag = ""
                if "không chính xác" in reason or "không tồn tại" in reason:
                     potential_citation = clause_content # Cần logic tốt hơn để trích dẫn
                     query_for_rag = f"Xác minh tính chính xác của trích dẫn pháp luật sau đây được nêu trong hợp đồng: '{potential_citation}'. Trích dẫn này có tồn tại không và nội dung chính là gì theo luật Việt Nam?"
                else:
                     query_for_rag = f"Kiểm tra pháp lý cho nội dung hợp đồng sau: '{clause_content}'. Lý do cần kiểm tra: '{reason}'. Cung cấp thông tin từ luật Việt Nam liên quan để đối chiếu."

                print(f"--- Querying RAG for issue related to: '{clause_content}' ---")
                response = await agent.aquery(query_for_rag)

                # <<< CHANGE: Phân tích kết quả RAG và định dạng output >>>
                relevant_response = response.response
                citation_text = "Không có nguồn tham khảo cụ thể được tìm thấy."
                rag_status = "unknown"

                if response.source_nodes:
                    citations = defaultdict(set)
                    valid_citation_found = False
                    # Chuẩn bị tên file luật để ưu tiên citation
                    cited_laws_files = [f.split('.')[0] + ".pdf" for f in cited_laws_list if isinstance(f, str)] # Đảm bảo là list string và có .pdf

                    for node in response.source_nodes:
                        metadata = node.node.metadata
                        page_number = metadata.get("page_label", "N/A")
                        file_name = metadata.get("file_name", "Unknown file")
                        if file_name in cited_laws_files: # Ưu tiên luật đã trích dẫn
                            citations[file_name].add(page_number)
                            valid_citation_found = True
                        # Cân nhắc thêm logic chấp nhận nguồn khác nếu cần

                    if valid_citation_found:
                        citation_text = "Nguồn tham khảo: " + "; ".join(
                             f"{file} (trang {', '.join(sorted(list(pages)))})"
                             for file, pages in citations.items() if pages # Chỉ thêm nếu set không rỗng
                         )
                        if not citation_text.strip().endswith(")"): # Nếu không có citation hợp lệ nào được thêm
                            citation_text = "Nguồn tham khảo được tìm thấy nhưng không khớp với luật trích dẫn trong HĐ."


                    # Đánh giá nội dung phản hồi của RAG
                    response_lower = relevant_response.lower() if relevant_response else ""
                    if "không tìm thấy thông tin" in response_lower or \
                       "không có điều khoản" in response_lower or \
                       "không tồn tại" in response_lower or \
                       "không xác định" in response_lower:
                        rag_status = "citation_not_found"
                    elif reason and ("mâu thuẫn" in reason or "không nhất quán" in reason or "diễn giải sai" in reason):
                        rag_status = "potential_contradiction"
                    else:
                        rag_status = "found_info"
                else: # Không có source node
                     response_lower = relevant_response.lower() if relevant_response else ""
                     if "không tìm thấy" in response_lower or "không có thông tin" in response_lower:
                         rag_status = "citation_not_found"


                # Thêm kết quả đã định dạng
                legal_issues_results.append(f"**Điểm cần lưu ý:** {clause_content}")
                legal_issues_results.append(f"  * **Lý do kiểm tra (từ LLM):** {reason}")

                if rag_status == "citation_not_found":
                    legal_issues_results.append(f"  * **Kết quả tra cứu (từ RAG):** {relevant_response}. **=> Cần kiểm tra lại tính chính xác của nội dung/trích dẫn này.**")
                elif rag_status == "potential_contradiction":
                    legal_issues_results.append(f"  * **Thông tin tra cứu (từ RAG):** {relevant_response}")
                    legal_issues_results.append(f"  * **Đánh giá:** Thông tin tra cứu được có vẻ khác biệt so với nội dung/trích dẫn trong hợp đồng. **=> Cần đối chiếu kỹ lưỡng.**")
                elif rag_status == "found_info":
                     legal_issues_results.append(f"  * **Thông tin tra cứu (từ RAG):** {relevant_response}")
                     legal_issues_results.append(f"  * **Đánh giá:** Đã tìm thấy thông tin liên quan, cần người dùng xem xét sự phù hợp.")
                else: # unknown or other cases
                     legal_issues_results.append(f"  * **Thông tin tra cứu (từ RAG):** {relevant_response}")

                legal_issues_results.append(f"  * **{citation_text}**")
                legal_issues_results.append("---")

            if not legal_issues_results:
                 final_legal_issues_output = "LLM xác định có điểm cần kiểm tra nhưng quá trình tra cứu không thành công hoặc không tìm thấy thông tin phù hợp."
            else:
                 final_legal_issues_output = "\n".join(legal_issues_results)

        print("------------------------------------------------------")
        print("Legal Issues Analysis (Post-RAG):")
        print(final_legal_issues_output)
        print("------------------------------------------------------")

        ctx.write_event_to_stream(ProgressEvent(progress="Detailed analysis completed."))
        return AnalysisResultsEvent(
            legal_issues=final_legal_issues_output,
            logic_issues=logic_issues_text,
            risk_issues=risk_issues_text
        )

    @step()
    async def synthesize_output(self, ctx: Context, ev: AnalysisResultsEvent) -> StopEvent:
        # <<< CHANGE: Lấy đúng biến context đã cập nhật >>>
        contract_type = await ctx.get("contract_type")
        key_elements = await ctx.get("key_elements")
        cited_laws = await ctx.get("cited_laws")
        contract_text = await ctx.get("contract_text")
        query = await ctx.get("query")

        context_info = {
            "contract_type": contract_type,
            "key_elements": key_elements,
            "cited_laws": cited_laws
        }
        # <<< CHANGE: Định nghĩa lại message này để dùng trong prompt >>>
        no_issues_found_msg = "Không xác định được điểm cụ thể nào trong văn bản cần đánh giá pháp lý thêm."

        # <<< CHANGE: Cập nhật prompt để xử lý format legal_issues mới và context mới >>>
        prompt = f"""
        **Yêu cầu:** Tổng hợp thông tin được cung cấp dưới đây để tạo báo cáo phân tích và mô tả chú thích. **Tuyệt đối chỉ sử dụng thông tin đã được cung cấp trong các mục 1, 2, 3.** Không thêm thông tin, đánh giá, hoặc suy diễn mới. Trình bày rõ ràng các phát hiện và kết quả tra cứu (bao gồm cả trường hợp không tìm thấy hoặc không có nguồn).

        **1. Thông tin Ngữ cảnh (Đầu vào):**
           - Loại hợp đồng: {context_info['contract_type']}
           - Yêu cầu kiểm tra ban đầu: {query}
           - Các yếu tố chính đã xác định: {context_info['key_elements']}
           - Văn bản luật được trích dẫn trong HĐ: {context_info['cited_laws']}

        **2. Kết quả Phân tích Chi tiết (Đầu vào):**
           - Phân tích Pháp lý (Kết quả kiểm tra các điểm đáng ngờ):
             ```text
             {ev.legal_issues}
             ```
           - Phân tích Logic & Ngữ nghĩa (JSON):
             ```json
             {ev.logic_issues}
             ```
           - Phân tích Rủi ro (JSON):
             ```json
             {ev.risk_issues}
             ```

        **3. Nội dung Hợp đồng Gốc (Để tham chiếu vị trí):**
        ---
        {contract_text}
        ---

        **Đầu ra mong muốn (Dựa hoàn toàn vào đầu vào):**

        **A. Báo cáo Phân tích Hợp đồng:**
           - **Tóm tắt:** Nêu bật các phát hiện chính từ Phân tích Pháp lý, Logic, và Rủi ro. Nếu `ev.legal_issues` là "{no_issues_found_msg}", hoặc `logic_issues`/`risk_issues` là JSON rỗng/không có phát hiện, hãy ghi rõ "Không có phát hiện đáng kể về mặt pháp lý/logic/rủi ro dựa trên phân tích tự động."
           - **Phân tích Pháp lý:**
             *   Nếu `ev.legal_issues` là "{no_issues_found_msg}", ghi: "{no_issues_found_msg}".
             *   Nếu khác, trình bày lại từng **Điểm cần lưu ý** từ `ev.legal_issues`, bao gồm **Lý do kiểm tra**, **Kết quả/Thông tin tra cứu**, **Đánh giá (nếu có)**, và **Nguồn tham khảo** đi kèm. **Trình bày y nguyên các thông báo 'không tìm thấy nguồn', 'cần kiểm tra lại', 'cần đối chiếu kỹ lưỡng' hoặc kết quả tra cứu.**
           - **Phân tích Logic & Ngữ nghĩa:** Trình bày các phát hiện từ `ev.logic_issues` (phân tích JSON). Nếu không có `logic_findings`, ghi "Không phát hiện vấn đề logic hay ngữ nghĩa nào từ văn bản."
           - **Phân tích Rủi ro:** Trình bày các phát hiện từ `ev.risk_issues` (phân tích JSON). Nếu không có `risk_findings`, ghi "Không phát hiện rủi ro cụ thể nào dựa trên các tiêu chí phân tích."
           - **Đề xuất chung:** Chỉ tổng hợp các đề xuất (`suggestion`) từ các phát hiện trong `ev.risk_issues`. **Không đưa ra đề xuất pháp lý trừ khi nó là kết quả trực tiếp từ thông tin tra cứu trong `ev.legal_issues`.** Không thêm đề xuất mới.

        **B. Mô tả Chú thích Hợp đồng (Dạng Markdown):**
           Đối với mỗi phát hiện có ý nghĩa trong `ev.legal_issues` (không phải câu thông báo chung), `ev.logic_issues` (từ `logic_findings`), `ev.risk_issues` (từ `risk_findings`), tạo một mục chú thích:
           ```markdown
           **[Ước tính vị trí: Dựa trên 'Điều khoản/Nội dung HĐ' hoặc mô tả vấn đề]** - [Trích dẫn/Mô tả ngắn gọn phần văn bản gốc có vấn đề]
           *   **Vấn đề:** [Mô tả vấn đề từ phát hiện (pháp lý/logic/rủi ro) và Lý do kiểm tra nếu là pháp lý]
           *   **Thông tin tham khảo / Đánh giá (nếu có):** [Trình bày 'Kết quả/Thông tin tra cứu', 'Đánh giá', và 'Nguồn tham khảo' từ ev.legal_issues nếu là vấn đề pháp lý và có kết quả, hoặc ghi 'N/A']
           *   **Đề xuất (nếu có):** [Đề xuất từ ev.risk_findings['suggestion'], hoặc ghi 'N/A']
           ---
           ```
        """
        # <<< CHANGE: Thay thế placeholder trong prompt >>>
        prompt = prompt.replace("{no_issues_found_msg}", no_issues_found_msg)

        output = await Settings.llm.acomplete(prompt)
        report = output.text
        print(report) # In ra để kiểm tra

        ctx.write_event_to_stream(ProgressEvent(progress="Synthesis completed."))
        # Chỉ trả về report theo định nghĩa event
        return StopEvent(result=FinalOutputEvent(report=report.strip()))

In [7]:
import fitz  # PyMuPDF
from docx import Document

def extract_text_from_pdf(pdf_path):
    """Trích xuất văn bản từ file PDF."""
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    return text

def extract_text_from_docx(docx_path):
    """Trích xuất văn bản từ file DOCX."""
    doc = Document(docx_path)
    text = "\n".join(paragraph.text for paragraph in doc.paragraphs)
    return text

def extract_text_from_txt(txt_path):
    """Trích xuất văn bản từ file TXT."""
    with open(txt_path, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

async def main(input_source: str, is_file_path: bool = False):
    contract_content = ""

    if is_file_path:
        try:
            if input_source.lower().endswith(".pdf"):
                contract_content = extract_text_from_pdf(input_source)
            elif input_source.lower().endswith(".docx"):
                contract_content = extract_text_from_docx(input_source)
            elif input_source.lower().endswith(".txt"):
                contract_content = extract_text_from_txt(input_source)
            else:
                raise ValueError("Unsupported file type. Use .pdf or .docx.")
        except Exception as e:
            print(f"Error processing file: {e}")
            contract_content = "Không thể đọc nội dung file."
    else:
        contract_content = input_source.strip()

    if contract_content:
        # print(contract_content)
        query = f"Kiểm tra hợp đồng lao động với nội dung sau theo luật Việt Nam:\n{contract_content}"
    else:
        query = "Kiểm tra hợp đồng lao động theo luật Việt Nam"

    workflow = MultiAgentContractReviewWorkflow(timeout=3000, verbose=True)
    # result = await workflow.run(ev=ContractFileEvent(file_path=contract_content, query="Kiểm tra hợp đồng lao động theo luật Việt Nam", tools=[law_tool]))
    result = await workflow.run(file_path=contract_content, query="Kiểm tra hợp đồng lao động theo luật Việt Nam", tools=[law_tool])
    print("Kết quả phân tích hợp đồng:")
    print(result)


In [8]:
if __name__ == "__main__":
    import asyncio
    
    # Example usage with file path (simulated due to Pyodide)
    # file_path = "/home/tuandatebayo/WorkSpace/Hop-dong-lao-dong.pdf"
    file_path = "/home/tuandatebayo/WorkSpace/law/dfcb9f0b-49eb-4665-b81d-f0956630531d.txt"
    asyncio.run(main(file_path, is_file_path=True))
    # await main(file_path, is_file_path=True)

    
    # Example usage with direct text input
    # text_input = """
    # Hợp đồng lao động
    # - Loại hợp đồng: Xác định thời hạn 1 năm
    # - Lương: 8 triệu VND/tháng
    # - Thử việc: 3 tháng
    # """
    # asyncio.run(main(text_input, is_file_path=False))

Running step preprocess_input
Step preprocess_input produced event ContractTextEvent
Running step analyze_context
{
  "contract_type": "Hợp đồng lao động không xác định thời hạn",
  "key_elements": [
    "Công việc: Chuyên viên Marketing",
    "Địa điểm làm việc: Văn phòng công ty tại 456 Đường Nguyễn Văn Cừ, Quận 5, TP. Hồ Chí Minh",
    "Thời hạn hợp đồng: Không xác định thời hạn",
    "Chế độ làm việc: 8 giờ/ngày từ 8h30 đến 17h30 (nghỉ trưa 1 giờ), 5 ngày/tuần",
    "Tiền lương chính: 18.000.000 VNĐ/tháng",
    "Phụ cấp ăn trưa: 600.000 VNĐ/tháng, Phụ cấp điện thoại: 400.000 VNĐ/tháng",
    "Hình thức trả lương: Chuyển khoản vào ngày 5 hàng tháng",
    "Tiền làm thêm giờ tối thiểu 150% tiền lương giờ làm việc bình thường",
    "Thưởng Tết ít nhất 1 tháng lương theo kết quả kinh doanh công ty",
    "Bên A cung cấp trang thiết bị làm việc và bảo hộ lao động",
    "Đảm bảo môi trường làm việc an toàn",
    "Bảo hiểm xã hội, bảo hiểm y tế, bảo hiểm thất nghiệp cho Bên B theo mức lương 